### Получение сборок.
1. Получение двух фаста файлов, списка всех UCE, файл со всеми ридами. 
   
   (cat IonXpress_\d\d\d_rawlib.basecaller_unadapt_trimmed0_0.cor.fasta > All_spicies.fasta)

    -> All_spicies.fasta
    -> All_UCE.fasta
    
2. Сборка de novo через Trinity. 

   (Trinity --seqType fa --single All_spicies.fasta) 
   
   -> All_spicies_Trinity.fasta
    
3. Поиск включений UCE в сборке через бласт (через биопитон?). 
   
   (makeblastdb  -in 'All_spicies_Trinity.fasta' -dbtype nucl -out blast_index, 
   
   blastn -query All_UCE.fasta -db blast_index -outfmt 6 -out UCE_in_Trinity.outfmt6)
   
   -> UCE_in_Trinity.outfmt6
   
4. Из UCE_in_Trinity.outfmt6 и All_spicies_Trinity.fasta составление фаста файлов с контигами, содержащими UCE, для каждого UCE.

   (with open UCE_in_Trinity.outfmt6:
     по имени UCE в первой строчке, создаем фаста файл, куда записываем из файла All_spicies_Trinity.fasta контиги, найденные в UCE_in_Trinity.outfmt6 по шаблону второй строки)
   
   -> UCE№_by_Trinity.fasta (по числу UCE)
   
5. Полученные файлы помещаю в CD-HIT-EST 
   
   (cd-hit-est -i UCE№_by_Trinity.fasta -c 0.8 -l 200 -r 1 -d 30)
   
6. Делаю фаста файлы с кластерами. 
   
   (в файле )
   
   -> UCE№_by_Trinity_Claster№.fasta (по числу кластеров для каждого UCE * число UCE)
   
7. Загружаю файлы с кластерами в сар3. 
   
   (cap3 UCE№_by_Trinity_Claster№.fasta  -p 85 -f 1500 -e 500 -z 1 -o 16 -s 251)
   
   -> UCE№_by_Trinity_cap3_Cosensus№.fasta
 
8. Для каждого консенсуса делаю сборку на него каждого из видов в BWA. 
   
   (bwa index UCE№_by_Trinity_cap3_Cosensus№.fasta
   
   bwa mem UCE№_by_Trinity_cap3_Cosensus№.fasta SP№.fasta > UCE№_SP№_Сonsensus№.sam
   samtoola view -F 4 UCE№_SP№_Сonsensus№.sam > UCE№_SP№_Сonsensus№_mapping.sam)
   
   -> UCE№_SP№_Сonsensus№_mapping.sam
   
9. Выделяю только те риды, которые выровнялись на референс

    samtools view -h -F 4 file.sam -o file_cart.sam
    
    samtools sort file.sam -o file.bam
    
    samtools index file.bam

Два основных модуля. 
1. Получение сборок
2. Чистка сборок

### Чистка сборок (прописано для одной сборки, повторяем для всех)
Наверно через биопитон или через pysam

Что делать если рида в сборке всего два?
Call_bases
Очистить sam файл, тогда колонок будет одинакого. Убрать первую строку.

In [1]:
import pysam
import Bio
import pandas as pd
from Bio import SeqIO

In [2]:
# Загружаем файл, берем из него колонки с ридом, и позицией начала рида Флаги!
data = pd.read_csv('UCE4_sp3_trinity_cap3_contig1_cart_without_head.sam', sep = '\t',header=None, usecols=[0,3,9,5,1])
data.columns = ['Название рида','Flag','Начало рида', 'CIGAR','Рид']
data = data.loc[data['Flag'] != 2048] # чистка по флагам
data

,Название рида,Flag,Начало рида,CIGAR,Рид
0,IonXpress_unpaired_ID373,16,138,72M,AATTAATGATCATGAGCTCGTATTTGATGGACTCTAGCTACATCGA...
1,IonXpress_unpaired_ID383,0,113,143M,TAGAAAAATTATTTTCCACTCCAGAAATTAATGATCATGAGCTCGT...
2,IonXpress_unpaired_ID696,0,77,82M,GTAGGAGGGCTTTATGGAGCAGAAAAACGACAAAGCTAGAAAAATT...
3,IonXpress_unpaired_ID1271,0,113,162M6I35M,TAGAAAAATAAATTTCCTCTGCAGAAATTAATGATCATGAGCTCGT...
4,IonXpress_unpaired_ID2237,0,35,171M,GCTGTGTCACGTGGTTAAGTAACTTTACAGGGTCGCTTGCTAGTAG...
...,...,...,...,...,...
1213,IonXpress_unpaired_ID1168776,16,282,52M,ATCACCACCAGGAGTTGTATCATCCGCGAGTGACCTACCAGGAGCG...
1214,IonXpress_unpaired_ID1170430,0,106,213M,ACAAAGGTAGAAAAAATATTTTTCACTCCAAAAATTAATGATCATG...
1215,IonXpress_unpaired_ID1170899,16,51,3S151M,TTAAAGTAACTTTACAGGGTCTCTTGCTAGTAGGAGGTCTTTATGG...
1216,IonXpress_unpaired_ID1171971,16,134,17S126M14S,AAAATAAATTTCCTCTGCAGAAATTAATGATCATGAGCTCGTATTT...


In [3]:
# загрузим соответствующий референс
ref_fasta= SeqIO.parse(open('UCE4_All_Trinity1.fasta'),'fasta')
for i in ref_fasta:
    bio_ref = i.seq
ref =[]
for i in bio_ref:
    ref.append(i)
#ref

In [4]:
def working_cigar(cigar):
    '''For input the cigar string, for output the cigar dictionary'''
    cigar = list(cigar.strip()) #сделали список
    # для начала нужно сигар разбить на блоки и переделать цифры в инт. По идее олжно получиться int(61) str(S) int(37) str(M)...
    # Каждый символ пытаюсь перевести в инт, обработка исключений на буквы
    for i in range(len(cigar)): #пробуем каждый символ списка сделать интом
        try:
            cigar[i] = int(cigar[i])
        except ValueError:
            cigar[i] = cigar[i]
            
    # Потом объединяю цифры в одно число.
    cigar_new = []
    for i in range(len(cigar)): # если символ - строка - записываем его отдельно
        if type(cigar[i]) == str:
            cigar_new.append(cigar[i])                                    
        try:
            if type(cigar[i]) == type(cigar[i+1]) == type(cigar[i+2]) : # если три символа подряд это инт, объединяем их
                cigar_new.append(int(str(cigar[i])+str(cigar[i+1])+str(cigar[i+2])))
            if type(cigar[i]) == type(cigar[i+1]) and type(cigar[i]) != type(cigar[i+2]) and type(cigar[i-1]) ==str:  # если два символа подряд одного типа (такое может быть только с инт), объединяем их  
                cigar_new.append(int(str(cigar[i])+str(cigar[i+1])))
            if type(cigar[i+1]) == str and type(cigar[i-1]) == str: # если один инт - тоже записываем его
                cigar_new.append(cigar[i])
        except IndexError: # это нужно на случай окончания строки, list index out of range
            continue
    #print(cigar_new) # проверка правильности составления списка из cigar      
    # удобнее сделать словарь
    numbers = cigar_new[::2] # список с числом нуклеотидов
    clipping_type = [] # список с типом совпадений
    for i in range(1, len(cigar_new), 2): 
        clipping_type.append(cigar_new[i]);
    #print(numbers, clipping_type) # проверка правильности списков
    cigar_dict = [] # делаем словарь
    for i in range(len(numbers)):
        cigar_dict.append((numbers[i],clipping_type[i]))
    
    return cigar_dict
#working_cigar(cigar)

In [5]:
def seq_after_cigar(cigar, seq):
    nucl_number = 0 # сразу столбец из data
    print(seq)
    for e in cigar:
        #print(key, value) # првоерка работы функции working_cogar
        if e[1] == 'S':
            seq = seq[:nucl_number] + 'N' * e[0] + seq[nucl_number + e[0]:]
        if e[1] == 'D':
            seq = seq[:nucl_number] + ' ' * e[0] + seq[nucl_number:]
        if e[1] == 'H':
            continue # это пока, может быть hard clipping не вырезается
        if e[1] == 'I':
            seq = seq[:nucl_number] + seq[nucl_number + e[0]:]
        nucl_number += e[0]
    #print(key, nucl_number)
    seq = seq.replace('N', '') 
    
    return seq
#print(seq_after_cigar(working_cigar(cigar), seq))

In [6]:
# попробуем теперь анализировать так все риды из таблицы.
new_data = []
for i in range(len(data)):
    new_data.append((data['Начало рида'].iloc[i] - 1) * ' ' + seq_after_cigar(working_cigar(data['CIGAR'].iloc[i]),data['Рид'].iloc[i]))

AATTAATGATCATGAGCTCGTATTTGATGGACTCTAGCTACATCGATCCGAAATTTCCTCCTTGCGAGGAAA
TAGAAAAATTATTTTCCACTCCAGAAATTAATGATCATGAGCTCGTATTTGATGGACTCTACCTACATTGATCCGAAATTTCCTCCATGCGAGGAATATTCGCAAAACAGCTACATCCCCGAGCACAGCCCCGAGTATTACAG
GTAGGAGGGCTTTATGGAGCAGAAAAACGACAAAGCTAGAAAAATTATTTTCCACTCCAGAAATTAATGATCATGAGCTCGT
TAGAAAAATAAATTTCCTCTGCAGAAATTAATGATCATGAGCTCGTATTTGATGGACTCTACCTACATTGATCCGAAATTTCCTCCATGCGAGGAATATTCGCAAAACAGCTACATCCCCGAGCACAGCCCCGAATATTACAGCCGCGCAAGGGACCCTGGCTCTGGTTACCAGCATCACCATCAGGAGTTATACCCTCCGCG
GCTGTGTCACGTGGTTAAGTAACTTTACAGGGTCGCTTGCTAGTAGGAGGGTTTTATGGACCAGAAAAACGACAAAGGTAGAAAAATTATTTTTCACTCCAGAAATTAATGATCATGAGCTCGTATTTGATGGACTCTAGCTACATCGATCCGAAATTTCCTCCTTGCGAG
GTTCCTTATCCGGGAACTACCTCGCCAGCTGCCATTGGACTGCTGTGTCACGTGGTTAAGTAACTTTACAGGGTCGCTTGCTAGTAGGAGGGTTTTATGGACCAGAAAAACGACAAAGGTAGAAAAATTATTTTTCACTCCAGAAATTAATGATCATGAG
CAGAAATTAATGATCATGAGCTCGTATTTGATGGACTCTACCTACATTGATCCGAAATTTCCTCCATGCGAGGAATATTCGCAAAACAGCTACATCCCCGAGCACAGCCCCGAATATTACAGCCGCGCAAGGGACCCTGGCTCTGGTTACCAGCATCACCATC

In [7]:
ref_cover_list = [] # большой словарь для всех нуклеотидов, по всей длине референса
for i in range(len(ref) + 1):
    cover_dict = {} # для каждого нуклеотида
    cover_dict['A, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['C, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['G, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['T, нуклеотид № {0:.0f}'.format(i+1)] = 0
    ref_cover_list.append(cover_dict)

for read in new_data:
    for nucl_number, nucl in enumerate(read):
            #print(nucl, nucl_number)
            if nucl == 'A':
                ref_cover_list[nucl_number]['A, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
            if nucl == 'T':
                ref_cover_list[nucl_number]['T, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
            if nucl == 'G':
                ref_cover_list[nucl_number]['G, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
            if nucl == 'C':
                ref_cover_list[nucl_number]['C, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
#ref_cover_list

# посчитаем покрытие для каждого нуклеотида
cover_list = [] # большой словарь для всех нуклеотидов, по всей длине референса
for i in range(len(ref) + 1):
    cover_dict = {} # для каждого нуклеотида
    cover_dict['A, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['C, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['G, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['T, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_list.append(cover_dict)


for nucl_number, nucl in enumerate(ref_cover_list):
    cover = 0
    for key, value in nucl.items():
        cover += value
    for key, value in nucl.items():    
        if key == 'A, нуклеотид № {:.0f}'.format(nucl_number + 1):
            if cover == 0:
                cover_list[nucl_number][key] =  0
            else:
                cover_list[nucl_number][key] =  value * 100 / cover
        if key == 'T, нуклеотид № {:.0f}'.format(nucl_number + 1):
            if cover == 0:
                cover_list[nucl_number][key] =  0
            else:
                cover_list[nucl_number][key] =  value * 100 / cover
        if key == 'G, нуклеотид № {:.0f}'.format(nucl_number + 1):
            if cover == 0:
                cover_list[nucl_number][key] =  0
            else:
                cover_list[nucl_number][key] =  value * 100 / cover
        if key == 'C, нуклеотид № {:.0f}'.format(nucl_number + 1):
            if cover == 0:
                cover_list[nucl_number][key] =  0
            else:
                cover_list[nucl_number][key] =  value * 100 / cover
                
    #print(nucl_number, cover)
#cover_list     

In [8]:
# Сравниваю каждый рид с референсом, выписываю замены в стиле:  [(53, 'A'), (100, 'T')...]
changing = []
for nucl_number, nucl in enumerate(ref): # для всех нуклеотидов референса
    for seq_number, seq in enumerate(new_data): # для каждого рида сборки
        #print(seq[nucl_number])
        try:
            if seq[nucl_number] != ' ': # если нуклеотид рида существует в этой позиции
                if nucl != seq[nucl_number]: # если нуклеотид референса не равен нуклеотиду рида
                    #print(seq_number)
                    changing.append([data['Название рида'].iloc[seq_number], 
                                     '{0:s}, нуклеотид № {1:.0f}'.format(seq[nucl_number], nucl_number +1)])                 
        except IndexError:
            continue
#for i in range(len(changing)):
#    print(changing[i])
# из этих данных нужно полуить информацию о заменах в каждом риде, сначала выбираю отсюда элементы для рида 9. Потом смотрю при каких номерах нуклеотидов какой тип замены.

In [9]:
# Отсюда можем получить отдельный набор для замен в каждом риде, и для замен по каждой позиции
# Посмотрим, в каких ридах встречаются замены и сколько в каждом риде замен.
read_with_change = []
for e in changing:
    read_with_change.append(e[0])

change = []
for read in read_with_change:
    change.append((read, read_with_change.count(read)))
read_with_change = set(change) # посмотрели в каких ридах есть замены.
#read_with_change

In [10]:
nucleotide_with_change = []
for e in changing:
    nucleotide_with_change.append(e[-1])

change = []
for read in nucleotide_with_change:
    change.append(('В {0:s}'.format(read), nucleotide_with_change.count(read)))

nucleotide_with_change = list(set(change))
nucleotide_with_change = sorted(nucleotide_with_change, key=lambda a: a[0][-3:])
#nucleotide_with_change

## Задача: убрать из сборки риды, которые не принадлежат этой хромосоме.
Как это понять?
1. В этих ридах есть инсерции и делеции
2. Для этих ридов большое количество замен: 10-15 (вариабельный параметр)
3. Смотрм для скольки процентов ридов характена такая замена (< 80 - вариабельный параметр)
4. Этот процент (80) умножаем на коэффициент покрытия в данном районе.

        Как считаем этот коэффициент?
        1. Смотрим на абсолютное покрытие в данном нуклеотиде (50)
        2. Смотрим на максимальное покрытие по всему референсу. (500)
        3. Смотрим для скольки нуклеотидов характерно максимальное покрытие в процентах (например для 100/400 и есть максимальное покрытие
        в 500 ридов.)
        4. Тогда, если для 25% референса максимальное покрытие 500 - коэффициент: 50 * 4/500
        
        0.8*50 - сколько ридов имеют замену в этом месте.
        500/ 50 - во сколько раз отличается покрытие в разных регионах
        
5. Умножаем процент замены на этот коэффициент. (0.8 * 50)*4/500 = 0.32. 
6. Если это число меньше 0.5 (вариабельный параметр),  увеличиваем счетчик для данного рида.
7. Для каждого рида, смотрим на число замен (20) и показания счетчика (8).
8. Если 'редких' замен (8) больше чем 40% (вариабельный параметр) (в данном случае как раз 40), то рид удаляется.

Поститать для данной сборки максимальное покрытие относительно длины референса.

In [13]:
candidate_for_remove = [] # список кандидатов на удаление
for read_tuple in read_with_change: # для всех ридов с заменами:
    if read_tuple[1] >= 10: # Более скольки замен должно быть в риде
        candidate_for_remove.append(read_tuple)
    # те риды, где есть инсерция и делеция, тоже попадают в список
    if 'I' in data.loc[data['Название рида'] == read_tuple[0]]['CIGAR'].iloc[0] or 'D' in data.loc[data['Название рида'] == read_tuple[0]]['CIGAR'].iloc[0]:
        candidate_for_remove.append(read_tuple)
candidate_for_remove = list(set(candidate_for_remove)) # сет, чтобы один рид не попал дважды

count =[]
for read_number, read in enumerate(candidate_for_remove): # в каждом риде с большим числом замен
    k = 0
    for i, e in enumerate(changing): # иду по списку всех возможных замен
        if read[0] == e[0]: # если совпадает название рида с ошибкой и название кандидата на удаление
            for cover in cover_list: # смотрим словарь покрытия
                for key, value in cover.items(): # key - тип нуклеотида с заменой и его номер, value - процент покрытия для этого типа замены
                    if key == e[1]: # когда нуклеотид совпадает с нуклеотидом из списка замен
                        if value < 40: # для скольки процентов ридов характнрна такая замена
                            # тут нужо сомтерть не отновительно всех редких замен, а и на покрытие. для этой зоны.
                            #print('Число замен в риде:', read[1],'\n', 'Информация о риде и замене:',e, '\n','Процент такой замены в целом:',value)
                            k+=1
    if k !=0:
        count.append((read,k)) # сколько в риде всего замен, и сколько из них "Редкие"
reads_for_remove = []
for i in count:
    if i[1] / i[0][1] > 0.4: #если относительно всех замен рида более 40% из них редкие,
        reads_for_remove.append(i[0][0])
        
len(sorted(reads_for_remove))
# подумать над тем, что для замены в нуклеотиде с малым покрытием бралось не 50% а 50% * отношнение абсолютного покрытия в этой точке на максимальное абсолютное покрытие

108

In [16]:
# Сделаем список с хорошими ридами
good_reads = []
for read in data['Название рида']:
    if read not in reads_for_remove:
        good_reads.append(read + '\t')
len(good_reads)

1109

In [39]:
with open("UCE4_sp3_trinity_cap3_contig1_cart.sam", "r") as file:
    with open("UCE4_sp3_trinity_cap3_contig1_cart_copy.sam", 'w') as output:
        line = file.readline()
        output.write(line)
        line = file.readline()
        output.write(line)
        line = file.readline()
        for read in good_reads:
            if read in good_reads:
                output.write(line)
                print(line)
            line = file.readline()


IonXpress_unpaired_ID373	16	Contig1	138	60	72M	*	0	0	AATTAATGATCATGAGCTCGTATTTGATGGACTCTAGCTACATCGATCCGAAATTTCCTCCTTGCGAGGAAA	*	NM:i:1	MD:Z:71T0	AS:i:71	XS:i:0

IonXpress_unpaired_ID383	0	Contig1	113	60	143M	*	0	0	TAGAAAAATTATTTTCCACTCCAGAAATTAATGATCATGAGCTCGTATTTGATGGACTCTACCTACATTGATCCGAAATTTCCTCCATGCGAGGAATATTCGCAAAACAGCTACATCCCCGAGCACAGCCCCGAGTATTACAG	*	NM:i:11	MD:Z:15T45G6C17T14A5T11A2A8T2A2C5	AS:i:91	XS:i:0

IonXpress_unpaired_ID696	0	Contig1	77	60	82M	*	0	0	GTAGGAGGGCTTTATGGAGCAGAAAAACGACAAAGCTAGAAAAATTATTTTCCACTCCAGAAATTAATGATCATGAGCTCGT	*	NM:i:4	MD:Z:9T8C16G15T30	AS:i:62	XS:i:0

IonXpress_unpaired_ID1271	0	Contig1	113	60	162M6I35M	*	0	0	TAGAAAAATAAATTTCCTCTGCAGAAATTAATGATCATGAGCTCGTATTTGATGGACTCTACCTACATTGATCCGAAATTTCCTCCATGCGAGGAATATTCGCAAAACAGCTACATCCCCGAGCACAGCCCCGAATATTACAGCCGCGCAAGGGACCCTGGCTCTGGTTACCAGCATCACCATCAGGAGTTATACCCTCCGCG	*	NM:i:28	MD:Z:9T1T3T1A2C40G6C17T14A5T11A2A8T5C9A1G2A3A1C17C8G2T8	AS:i:81	XS:i:0

IonXpress_unpaired_ID2237	0	Contig1	35	60	171M	*	0	0	GCTGTGT